In [2]:
import pandas as pd
import polars as pl
import fun
import random
import numpy as np

## Data Prep

In [3]:
random.random()*(random.random()*0.1)

0.0018216018825589703

In [4]:
customer_lat = -6.2124234
customer_long = 106.816634

coord_center = (customer_long, customer_lat)

list_array_lat = []
list_array_long = []

coord_list = []

for i in range(120):
    N = random.randint(0,80)
    lat1 = []
    long1 = []
    for i in range(N):
        diff1 = random.random()*(random.random()*0.1)
        diff2 = random.random()*(random.random()*0.1)

        lat1.append(customer_lat+diff1)
        long1.append(customer_long+diff2)
    list_array_lat.append(lat1)
    list_array_long.append(long1)

for i in range(500):
    diff1 = random.random()*(random.random()*0.1)
    diff2 = random.random()*(random.random()*0.1)

    coord_list.append((customer_long+diff1, customer_lat+diff2))




In [5]:
df = pd.DataFrame()
df['long_array'] = list_array_long
df['lat_array'] = list_array_lat
df.head()

,long_array,lat_array
0,"[106.83900612358858, 106.89387114663545, 106.8...","[-6.146572947484051, -6.20730176565767, -6.181..."
1,"[106.83622276808306, 106.82060183757042, 106.8...","[-6.194823170734209, -6.205283760977219, -6.18..."
2,"[106.83463289676739, 106.83503781094517, 106.8...","[-6.212110750549248, -6.17764282651976, -6.164..."
3,"[106.84756071376047, 106.89653405206164, 106.8...","[-6.20716146392818, -6.18448988662932, -6.1991..."
4,"[106.82178988918174, 106.86278594505235, 106.8...","[-6.174723990378365, -6.195635983286176, -6.16..."


## Optimize Harvendistance

In [5]:
coord_neigh = np.array(coord_list[30:31])

In [6]:
fun.haversine_dist1(coord_neigh[0],coord_center) == np.round(fun.haversine_dist2(coord_neigh[:,0], coord_neigh[:,1], coord_center),5)[0]

True

In [7]:
coord_neigh_list1 = np.array(coord_list)
coord_neigh_list2 = np.array(coord_list)

In [8]:
%%timeit -n 500 -r 10
dist_list1 = []
for coord_n in coord_neigh_list1:
    xx = fun.haversine_dist1(coord_n,coord_center)
    dist_list1.append(xx)
# dist_list1

1.57 ms ± 25.8 µs per loop (mean ± std. dev. of 10 runs, 500 loops each)


In [12]:
%%timeit -n 500 -r 10
dist_list2 = np.round(fun.haversine_dist2(coord_neigh_list2[:,0], coord_neigh_list2[:,1], coord_center),5)

30.7 µs ± 1.17 µs per loop (mean ± std. dev. of 10 runs, 500 loops each)


In [13]:
# dist_list1[:10]

In [14]:
# dist_list2[:10]

In [15]:
# np.radians([-6.12])

In [16]:
# %%timeit -n 10000 -r 10
# fun.haversine_dist1(coord_neigh,coord_center)

# %%timeit -n 10000 -r 10
# fun.haversine_dist2(coord_neigh,coord_center)

## Optimize the loop

In [6]:
%%timeit
distance_summary = [fun.haversine_distance_summary(
                    zip(df['long_array'].loc[i], df['lat_array'].loc[i]),
                    coord2 = coord_center) 
                for i in range(len(df))]

df['min_dist_to_other_dest'] = [one_row[0] for one_row in distance_summary]
df['avg_dist_to_other_dest'] = [one_row[1] for one_row in distance_summary]

15.3 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
df['hdist_summary'] = df.apply(lambda x: fun.hdist_summary(x['long_array'], x['lat_array'], coord_center), axis=1)
df[['min_dist_to_other_dest2', 'avg_dist_to_other_dest2']] = pd.DataFrame(df['hdist_summary'].tolist(), index=df.index)

4.87 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# df